In [ ]:
!pip install matchms
!pip install rdkit
!pip install torch_geometric
!pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.1/235.1 kB 7.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 37.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 11.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910476 sha256=73e2a91b1e9df6998e10d95e3e41e6f079cef4eef9b2bec77dbd7efa7451c4ca
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dc

In [ ]:
from matchms.importing import load_from_msp
import numpy as np
import os
import random
import pickle
from rdkit import Chem
from rdkit.Chem import Descriptors
import matchms
from matchms import Spectrum

import matplotlib.pyplot as plt
import warnings

from rdkit.Chem.rdmolops import GetAdjacencyMatrix

# Pytorch and Pytorch Geometric
import torch
from torch_geometric.data import Data
from torch.utils.data import DataLoader

import torch
from torch.nn import Linear
import torch.nn.functional as F 
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool, GATConv
from torch_geometric.nn import global_mean_pool as gap
import torch.nn as nn
from torch_geometric.data import DataLoader

This notebook is a preprocessing step for graph neural networks as they must be specially handled.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/MyDrive/NIST_SMALL")
BASE_DIRECTORY = "/content/drive/MyDrive/NIST_SMALL"

TEST_DATA_SIZE = 5000
OUTPUT_SIZE = 1000
INTENSITY_POWER = 0.5

Here is the dataset that we used after RASSP sorting.

In [ ]:
nist_dataset_org_test = pd.read_parquet('/content/drive/MyDrive/NIST_SMALL/smalltest.pq')
nist_dataset_org_train = pd.read_parquet('/content/drive/MyDrive/NIST_SMALL/smalltrain.pq')

In [ ]:
nist_dataset_org_test

,rdmol,spect,smiles
8,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[37.0, 14.99], [38.0, 57.95], [39.0, 333.7], ...",CC1=NN(C(=O)C1(C)C)C2=CC=CC=C2
19,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[15.0, 4.0], [26.0, 6.99], [27.0, 99.91], [28...",CCC(C(C)CC(C)C(=O)CC)O
20,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[25.0, 7.99], [26.0, 88.92], [27.0, 146.87], ...",CC=NC(CC#N)C#N
26,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[24.0, 1.0], [25.0, 4.0], [26.0, 30.97], [27....",C1=CC(=NC(=C1)C=O)C=O
37,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[119.0, 79.93], [120.0, 819.26], [121.0, 79.9...",CCCCC1C(=O)N(C2=CC=CC=C2O1)O
...,...,...,...
197091,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[45.0, 37.97], [50.0, 27.97], [51.0, 55.95], ...",CC(C1=CC2=C(C=C1)OC(=N2)C3=CC=C(C=C3)Cl)C(=O)O
197093,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[17.0, 3.0], [18.0, 16.98], [27.0, 8.99], [28...",CCCC(=O)C1=CC(=C(C=C1O)O)O
197094,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[14.0, 1.0], [15.0, 12.99], [16.0, 2.0], [17....",COC1=CC=C(C=C1)C2(CC2)C(=O)O
197099,b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...,"[[41.0, 11.99], [43.0, 5.99], [44.0, 21.98], [...",C1=CC=C2C(=C1)C(=O)C(=C(C2=O)Cl)NC3=CC=C(C=C3)N


In [ ]:
len(nist_dataset_org_test)

28082

In [ ]:
len(nist_dataset_org_train)

112709

In [ ]:
i = 0

for index, row in test.iterrows():
    print(row)
    sample = row
    i += 1
  
    if i == 2:
        break

rdmol     b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...
spect     [[37.0, 14.99], [38.0, 57.95], [39.0, 333.7], ...
smiles                       CC1=NN(C(=O)C1(C)C)C2=CC=CC=C2
Name: 8, dtype: object
rdmol     b'\xef\xbe\xad\xde\x00\x00\x00\x00\r\x00\x00\x...
spect     [[15.0, 4.0], [26.0, 6.99], [27.0, 99.91], [28...
smiles                               CCC(C(C)CC(C)C(=O)CC)O
Name: 19, dtype: object


In [ ]:
i=0
for index, row in test.iterrows():
    print(row["smiles"])
    i += 1
  
    if i == 2:
        break

CC1=NN(C(=O)C1(C)C)C2=CC=CC=C2
CCC(C(C)CC(C)C(=O)CC)O


In [ ]:
def spectrum_preparation_double(spectrum, intensity_power, output_size, operation):
    # get spectrum object and return array of specific size for prediction
    # spectrum is in shape tuple[tuple[2]]
    spectrum_output = torch.zeros(1, output_size)
    
    
    for position, intensity in spectrum:
        if position >= output_size:
            spectrum_output[0][output_size - 1] = intensity
            continue
        spectrum_output[0][int(position)] = intensity
    
    if operation == "pow":
        spectrum_output = torch.pow(spectrum_output, intensity_power)
    elif operation == "log":
        spectrum_output = spectrum_output + 1
        spectrum_output = torch.log(spectrum_output)
    else:
        spectrum_output = spectrum_output
    
    return spectrum_output.type(torch.float64)

In [ ]:
def one_hot_encoding(label, num_labels):
    # make one hot encoding for one instance
    # args
    # label: int, position in one hot vector
    # num_label = int, how many groups exist
    # return: torch tensor
    tmp_zeroes = torch.zeros(num_labels)
    
    
    if type(label) is bool:
        tmp_zeroes[0] = label
        return tmp_zeroes
    if label >= num_labels:
        tmp_zeroes[num_labels - 1] = float(1)
        warnings.warn("Number of group is greater than one hot dimension representation")
        return tmp_zeroes
    elif label < 0:
        tmp_zeroes[0] = float(1)
        return tmp_zeroes
    else:
        tmp_zeroes[label] = float(1)
    return tmp_zeroes

In [ ]:
def get_atom_features(atom):
#   Featurise one atom and store it into torch tensor
    torch_result = torch.tensor([])

    atomic_number = torch.tensor([atom.GetAtomicNum()]) / 100.0
    torch_result = torch.cat((torch_result, atomic_number), 0)

    PERMITTED_LIST_OF_ATOMAS =  ['H','C','N','O','F','P', 'S', 'Unknown']
    atom_dict = {elem: index for index, elem in enumerate(PERMITTED_LIST_OF_ATOMAS)}
    
    atom_type_hot = one_hot_encoding(atom_dict.get(atom.GetSymbol(), len(atom_dict)),
                                     len(PERMITTED_LIST_OF_ATOMAS))

    torch_result = torch.cat((torch_result, atom_type_hot), 0)
    
    total_valence = atom.GetTotalValence()
    total_valence_hot = one_hot_encoding(total_valence, 8)
    torch_result = torch.cat((torch_result, total_valence_hot), 0)
    
    is_aromatic_hot = one_hot_encoding(atom.GetIsAromatic(), 1)
    torch_result = torch.cat((torch_result, is_aromatic_hot), 0)
    
    
    HYBRIDIZATIONS = [Chem.HybridizationType.UNSPECIFIED,
                      Chem.HybridizationType.S, 
                      Chem.HybridizationType.SP, 
                      Chem.HybridizationType.SP2, 
                      Chem.HybridizationType.SP3, 
                      Chem.HybridizationType.SP3D, 
                      Chem.HybridizationType.SP3D2,
                      Chem.HybridizationType.OTHER]
    hybridization_dict = {elem: index for index, elem in enumerate(HYBRIDIZATIONS)}
    hybridization = atom.GetHybridization()
    hybridization_hot = one_hot_encoding(hybridization_dict.get(hybridization, len(hybridization_dict)), 8)
    torch_result = torch.cat((torch_result, hybridization_hot), 0)
    
    
    # we adapt scale, the output of method GetFormalCharge is [-2, -1, 0, 1, 2]
    formal_charge = atom.GetFormalCharge()
    formal_charge_hot = one_hot_encoding(formal_charge + 2, 5)
    torch_result = torch.cat((torch_result, formal_charge_hot), 0)
    
    default_valence = Chem.GetPeriodicTable().GetDefaultValence(atom.GetAtomicNum())
    default_valence_hot = one_hot_encoding(default_valence, 8)
    torch_result = torch.cat((torch_result, default_valence_hot), 0)
    
    ring_size = [atom.IsInRingSize(r) for r in range(3, 8)]
    ring_size_hot = torch.tensor(ring_size).type(torch.float)
    torch_result = torch.cat((torch_result, ring_size_hot), 0)
    
    attached_H = np.sum([neighbour.GetAtomicNum() == 1 for neighbour in atom.GetNeighbors()], dtype=np.uint8)
    explicit = atom.GetNumExplicitHs()
    implicit = atom.GetNumImplicitHs()
    H_num = attached_H + explicit + implicit

    try:
        H_hot = one_hot_encoding(H_num, 6)
    except:
        print(H_num)
        print(attached_H, explicit, implicit)
        raise Exception("Sorry, no numbers below zero") 
        

    torch_result = torch.cat((torch_result, H_hot), 0)

    return torch_result
            
    


In [ ]:
# TEST PRINT

# sample = nist_dataset_org_train.iloc[3]['smiles']
# sample_mol = Chem.MolFromSmiles(sample)

# i = 0
# for atom in sample_mol.GetAtoms():
#     sample_atom = atom
# #     print(sample_atom.GetTotalValence() == Chem.GetPeriodicTable().GetDefaultValence(sample_atom.GetAtomicNum()))
# #     print(Chem.GetPeriodicTable().GetDefaultValence(sample_atom.GetAtomicNum()))
#     a = [sample_atom.IsInRingSize(r) for r in range(3, 8)]
#     print(torch.tensor(a).type(torch.float))
#     print(get_atom_features(sample_atom))
#     break
# for bond in sample_mol.GetBonds():
#     sample_bond = bond
#     break

tensor([0., 0., 0., 0., 0.])
tensor([0.0600, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000])


In [ ]:
# nist_dataset_org_train.iloc[3]['smiles']

'CC1=NN=C(O1)CCC(=O)O'

In [ ]:
def get_bond_features(bond, use_stereochemistry = True):  
# Takes an RDKit bond object as input and gives a 1d-numpy array of bond features as output.
        
    torch_result = torch.tensor([])
    
    BOND_TYPE = [1.0, 1.5, 2.0, 3.0]
    bond_dict = {elem: index for index, elem in enumerate(BOND_TYPE)}
    bond_type_hot = one_hot_encoding(bond_dict.get(bond.GetBondTypeAsDouble(), len(bond_dict)),
                                     len(BOND_TYPE))
    torch_result = torch.cat((torch_result, bond_type_hot), 0)
    
    bond_is_conj_hot = one_hot_encoding(bond.GetIsConjugated(), 1)
    torch_result = torch.cat((torch_result, bond_is_conj_hot), 0)
    
    bond_is_in_ring_hot = one_hot_encoding(bond.IsInRing(), 1)
    torch_result = torch.cat((torch_result, bond_is_in_ring_hot), 0)

    
    if use_stereochemistry == True:
        STEREO_TYPE = ["STEREOZ", "STEREOE", "STEREOANY", "STEREONONE"]
        stereo_dict = {elem: index for index, elem in enumerate(STEREO_TYPE)}
        stereo_type_hot = one_hot_encoding(stereo_dict.get(str(bond.GetStereo()), len(stereo_dict)),
                                                           len(STEREO_TYPE))
        torch_result = torch.cat((torch_result, stereo_type_hot), 0)
    return torch_result

In [ ]:
def create_pytorch_geometric_graph_data_list_parquet(nist_data, intensity_power, output_size, operation):
#     Inputs: 
#     Pandas dataframe with columns: 
#     rdkit mol
#     spectrum: tuple[tuple[2]]
#     smiles
#     Outputs:
#     data_list = list of torch_geometric.data.Data objects which represent-
#                 labeled molecular graphs that can readily be used for machine learning   
    
    data_list = []
    
    for _, nist_obj in nist_data.iterrows():
        
        # convert SMILES to RDKit mol object
        mol = Chem.MolFromSmiles(nist_obj['smiles'])
        
        if mol == None:            
            continue
       

        # get feature dimensions
        n_nodes = mol.GetNumAtoms()
        n_edges = 2*mol.GetNumBonds()
        
        # the purpose is to find out one hot emb dimension
        unrelated_smiles = "O=O"
        unrelated_mol = Chem.MolFromSmiles(unrelated_smiles)
        n_node_features = len(get_atom_features(unrelated_mol.GetAtomWithIdx(0)))
        n_edge_features = len(get_bond_features(unrelated_mol.GetBondBetweenAtoms(0,1)))

        # construct node feature matrix X of shape (n_nodes, n_node_features)
        X = np.zeros((n_nodes, n_node_features))

        for atom in mol.GetAtoms():
            X[atom.GetIdx(), :] = get_atom_features(atom)
            
        X = torch.tensor(X, dtype = torch.float64)
        
        # construct edge index array E of shape (2, n_edges)
        (rows, cols) = np.nonzero(GetAdjacencyMatrix(mol))
       
        torch_rows = torch.from_numpy(rows.astype(np.int64)).to(torch.long)
        torch_cols = torch.from_numpy(cols.astype(np.int64)).to(torch.long)
        E = torch.stack([torch_rows, torch_cols], dim = 0)
        
        # construct edge feature array EF of shape (n_edges, n_edge_features)
        EF = np.zeros((n_edges, n_edge_features))
        
        for (k, (i,j)) in enumerate(zip(rows, cols)):
            
            EF[k] = get_bond_features(mol.GetBondBetweenAtoms(int(i),int(j)))
        
        EF = torch.tensor(EF, dtype = torch.float)
        
        # weight of molecul
        MW = nist_obj.get("mw", None)
        if MW == None:
            MW = Descriptors.ExactMolWt(mol)
        MW = torch.tensor(int(round(float(MW))))
        
        # construct label tensor
        y_tensor = spectrum_preparation_double(nist_obj["spect"], intensity_power, output_size, operation)
        
        # construct Pytorch Geometric data object and append to data list
        data_list.append(Data(x = X, edge_index = E, edge_attr = EF, molecular_weight = MW, y = y_tensor))

    return data_list

In [ ]:
data_list_test = create_pytorch_geometric_graph_data_list_parquet(nist_dataset_org_test, INTENSITY_POWER, OUTPUT_SIZE, "pow")
data_list_test

<ipython-input-11-a6f8230a9932>:15: UserWarning: Number of group is greater than one hot dimension representation
  warnings.warn("Number of group is greater than one hot dimension representation")
[07:42:55] WARNING: not removing hydrogen atom without neighbors
[07:43:22] WARNING: not removing hydrogen atom without neighbors
[07:46:31] WARNING: not removing hydrogen atom without neighbors


[Data(x=[15, 50], edge_index=[2, 32], edge_attr=[32, 10], y=[1, 1000], molecular_weight=202),
 Data(x=[13, 50], edge_index=[2, 24], edge_attr=[24, 10], y=[1, 1000], molecular_weight=186),
 Data(x=[9, 50], edge_index=[2, 16], edge_attr=[16, 10], y=[1, 1000], molecular_weight=121),
 Data(x=[10, 50], edge_index=[2, 20], edge_attr=[20, 10], y=[1, 1000], molecular_weight=135),
 Data(x=[16, 50], edge_index=[2, 34], edge_attr=[34, 10], y=[1, 1000], molecular_weight=221),
 Data(x=[14, 50], edge_index=[2, 30], edge_attr=[30, 10], y=[1, 1000], molecular_weight=197),
 Data(x=[16, 50], edge_index=[2, 34], edge_attr=[34, 10], y=[1, 1000], molecular_weight=210),
 Data(x=[14, 50], edge_index=[2, 28], edge_attr=[28, 10], y=[1, 1000], molecular_weight=251),
 Data(x=[17, 50], edge_index=[2, 32], edge_attr=[32, 10], y=[1, 1000], molecular_weight=238),
 Data(x=[11, 50], edge_index=[2, 20], edge_attr=[20, 10], y=[1, 1000], molecular_weight=158),
 Data(x=[12, 50], edge_index=[2, 24], edge_attr=[24, 10], y=[

In [ ]:
data_list_train = create_pytorch_geometric_graph_data_list_parquet(nist_dataset_org_train, INTENSITY_POWER, OUTPUT_SIZE, "pow")
data_list_train

<ipython-input-11-a6f8230a9932>:15: UserWarning: Number of group is greater than one hot dimension representation
  warnings.warn("Number of group is greater than one hot dimension representation")
[07:49:57] WARNING: not removing hydrogen atom without neighbors
[07:50:02] WARNING: not removing hydrogen atom without neighbors
[07:51:53] WARNING: not removing hydrogen atom without neighbors
[07:52:31] WARNING: not removing hydrogen atom without neighbors
[07:52:52] WARNING: not removing hydrogen atom without neighbors
[07:54:17] WARNING: not removing hydrogen atom without neighbors
[07:54:24] WARNING: not removing hydrogen atom without neighbors
[07:54:46] WARNING: not removing hydrogen atom without neighbors
[07:56:28] WARNING: not removing hydrogen atom without neighbors
[07:56:50] WARNING: not removing hydrogen atom without neighbors
[07:58:52] WARNING: not removing hydrogen atom without neighbors
[08:07:00] WARNING: not removing hydrogen atom without neighbors
[08:07:48] WARNING: no

[Data(x=[12, 50], edge_index=[2, 24], edge_attr=[24, 10], y=[1, 1000], molecular_weight=163),
 Data(x=[8, 50], edge_index=[2, 16], edge_attr=[16, 10], y=[1, 1000], molecular_weight=110),
 Data(x=[18, 50], edge_index=[2, 40], edge_attr=[40, 10], y=[1, 1000], molecular_weight=240),
 Data(x=[11, 50], edge_index=[2, 22], edge_attr=[22, 10], y=[1, 1000], molecular_weight=156),
 Data(x=[27, 50], edge_index=[2, 58], edge_attr=[58, 10], y=[1, 1000], molecular_weight=362),
 Data(x=[16, 50], edge_index=[2, 34], edge_attr=[34, 10], y=[1, 1000], molecular_weight=204),
 Data(x=[11, 50], edge_index=[2, 22], edge_attr=[22, 10], y=[1, 1000], molecular_weight=172),
 Data(x=[15, 50], edge_index=[2, 30], edge_attr=[30, 10], y=[1, 1000], molecular_weight=206),
 Data(x=[20, 50], edge_index=[2, 42], edge_attr=[42, 10], y=[1, 1000], molecular_weight=272),
 Data(x=[12, 50], edge_index=[2, 24], edge_attr=[24, 10], y=[1, 1000], molecular_weight=165),
 Data(x=[10, 50], edge_index=[2, 20], edge_attr=[20, 10], y=[

The code is commented on to avoid unintentional rewriting of the created dataset. And this is the central preprocessed data we used to train our models.

In [ ]:

# PREPROCESSED_FILE = "/content/drive/MyDrive/NIST_SMALL/Preprocessed_test_pow_preparation_no_sparse_small.output"
# with open(PREPROCESSED_FILE, 'wb') as fid:
#   pickle.dump(data_list_test, fid)
#   fid.close() 

# PREPROCESSED_FILE = "/content/drive/MyDrive/NIST_SMALL/Preprocessed_train_pow_preparation_no_sparse_small.output"
# with open(PREPROCESSED_FILE, 'wb') as fid:
#   pickle.dump(data_list_train, fid)
#   fid.close() 